# Economic Evaluation and Sensitivity Analysis

This notebook demonstrates economic analysis capabilities including NPV calculations, cash flow analysis, and sensitivity studies.

## Learning Objectives

By the end of this notebook, you will be able to:
- Calculate NPV and economic metrics
- Perform sensitivity analysis
- Run price scenarios
- Evaluate economic performance

## Table of Contents
1. [Setup](#setup)
2. [Basic Economic Analysis](#basic)
3. [Price Scenarios](#scenarios)
4. [Sensitivity Analysis](#sensitivity)
5. [Portfolio Analysis](#portfolio)

## 1. Setup {#setup}

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from decline_curve import dca
from decline_curve.economics import economic_metrics
from decline_curve.scenarios import PriceScenario, run_price_scenarios
from decline_curve.sensitivity import run_sensitivity
from decline_curve.portfolio import aggregate_by_category
from decline_curve.plot import minimal_style

minimal_style()
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All imports successful!")

## 2. Basic Economic Analysis {#basic}

In [ ]:
# Generate production forecast
dates = pd.date_range('2020-01-01', periods=36, freq='MS')
production = pd.Series(1000 * np.exp(-0.01 * np.arange(36)), index=dates)
forecast = dca.forecast(production, model='arps', kind='hyperbolic', horizon=60)

# Economic parameters
oil_price = 60.0  # $/bbl
opex = 15.0  # $/bbl operating costs
discount_rate = 0.10  # 10% annual discount rate

# Calculate economic metrics
economics = economic_metrics(forecast.values, oil_price, opex, discount_rate)

print("=" * 70)
print("ECONOMIC ANALYSIS")
print("=" * 70)
print(f"NPV: ${economics['npv']:,.2f}")
print(f"IRR: {economics.get('irr', 0):.2%}")
print(f"Payback Period: {economics.get('payback', 0):.1f} months")
print(f"Cumulative Cash Flow: ${economics.get('cumulative_cash_flow', 0):,.2f}")
print("=" * 70)

## 3. Price Scenarios {#scenarios}

In [ ]:
# Define price scenarios
scenarios = [
    PriceScenario('base', oil_price=60.0, opex=15.0),
    PriceScenario('low', oil_price=40.0, opex=15.0),
    PriceScenario('high', oil_price=80.0, opex=15.0),
]

# Run scenarios
results = run_price_scenarios(forecast.values, scenarios, discount_rate=0.10)

print("=" * 70)
print("PRICE SCENARIO ANALYSIS")
print("=" * 70)
print(results[['scenario', 'npv', 'irr']].to_string(index=False))
print("=" * 70)

## 4. Sensitivity Analysis {#sensitivity}

In [ ]:
# Parameter grid for sensitivity
qi_values = [800, 1000, 1200]
di_values = [0.10, 0.15, 0.20]
b_values = [0.3, 0.5, 0.7]
prices = [50, 60, 70]

# Create parameter combinations
param_grid = [(qi, di, b) for qi in qi_values for di in di_values for b in b_values]

# Run sensitivity analysis
sens_results = run_sensitivity(param_grid, prices, opex=15.0, discount_rate=0.10)

print("=" * 70)
print("SENSITIVITY ANALYSIS")
print("=" * 70)
print(f"Total combinations: {len(sens_results)}")
print(f"\nTop 5 by NPV:")
print(sens_results.nlargest(5, 'NPV')[['qi', 'di', 'b', 'price', 'NPV']].to_string(index=False))
print("=" * 70)

## Summary

This notebook demonstrated:
- ✓ Economic metrics calculation (NPV, IRR, payback)
- ✓ Price scenario analysis
- ✓ Parameter sensitivity studies

### Next Steps
- Explore portfolio-level analysis
- Learn about Monte Carlo simulation for uncertainty
- Study risk reporting and reserves classification